In [38]:
import tensorflow as tf
import os
import cv2 #pip3 install opencv-python

from tensorflow import keras
import random
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from PIL import Image, ImageOps, ImageFilter, ImageEnhance  #PILLOW
from math import floor, ceil, pi
import numpy as np 
%matplotlib inline

In [39]:
#Funzione per fare mirror orizzonale delle immagini e delle relative maschere
            #immagine, nome immagine, maschera, salvataggio immagine, salvataggio maschera
def mirror_hor(img,name_img,mask,dest_img, dest_mask):
    flip_img = ImageOps.mirror(img)
    flip_mask = ImageOps.mirror(mask)
    dest_img = os.path.join(dest_img,"trasl_"+name_img)
    dest_mask = os.path.join(dest_mask,"trasl_"+name_img)
    flip_img.save(dest_img)
    flip_mask.save(dest_mask)

In [40]:
#Funzione per cambiare il colore di un immagine (la maschera rimane uguale)
def color(img,name_img,mask,dest_img,dest_mask):
    img = img.convert("RGBA")
    factor = random.uniform(0.0,3.0) # fattore random per la modifica del colore
    new_img = ImageEnhance.Color(img).enhance(factor)
    
    #Save
    dest_img = os.path.join(dest_img,"col_"+name_img)
    dest_mask = os.path.join(dest_mask,"col_"+name_img)
    new_img.save(dest_img)
    mask.save(dest_mask)

In [41]:
#Funzione per aggiungere rumore gaussiano alle immagini (le maschere rimangono uguali)
#Aggiunta di pixel di vari colori sull'immagine
def gaussian_noise(image_path,name_img,mask,dest_img,dest_mask):
    path = os.path.join("..","original-dataset","images",name_img)
    img = cv2.imread(path) #unico che funziona
   
    # Generate Gaussian noise
    gauss = np.random.normal(0,0.6,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
    # Add the Gaussian noise to the image
    img_gauss = cv2.add(img,gauss)
    
    # Display the image
    #cv2.imshow('a',img_gauss)
    #cv2.waitKey(0)
    path2 = os.path.join("..","car-segmentation","images","gauss_"+name_img)
    cv2.imwrite(path2,img_gauss)
    dest_mask = os.path.join(dest_mask,"gauss_"+name_img)
    mask.save(dest_mask)

In [42]:
#Funzione per sfocare o definire un'immagine in modo random (maschere uguali)
def blur(img,name_img,mask,dest_img,dest_mask):
    img = img.convert("RGBA")
    
    factor = random.uniform(-2.0,4.0) # fattore random per la modifica del colore
    new_img = ImageEnhance.Sharpness(img).enhance(factor)
    #new_img.show()
    
    #Save
    dest_img = os.path.join(dest_img,"blur_"+name_img)
    dest_mask = os.path.join(dest_mask,"blur_"+name_img)
    new_img.save(dest_img)
    mask.save(dest_mask)
    

In [43]:
#Funzione per ruotare di 25° o -25° (random) ogni immagine e ogni maschera
def rotate(img,name_img,mask,dest_img,dest_mask):
    angle =  random.choice([-25, 25])
    new_img = img.rotate(angle)
    new_mask = mask.rotate(angle,fillcolor=0) #Colore di background della maschera
    #new_mask.show()
    
    #Save
    dest_img = os.path.join(dest_img,"rot_"+name_img)
    dest_mask = os.path.join(dest_mask,"rot_"+name_img)
    new_img.save(dest_img)
    new_mask.save(dest_mask)
    

In [64]:

#Funzione per traslare di 10 pixel in alto o in basso (random) ogni immagine e ogni maschera
def traslate(img,name_img,mask,dest_img,dest_mask): 
    direction = random.choice([-10,10])
    new_img = img.transform(img.size, Image.AFFINE, (1, 0, 0, 0, 1, direction))
    new_mask = mask.transform(mask.size, Image.AFFINE, (1, 0, 0, 0, 1, direction))
    new_img.show()
    #new_mask.show()
    
    #Save
    dest_img = os.path.join(dest_img,"trasl_"+name_img)
    dest_mask = os.path.join(dest_mask,"trasl_"+name_img)
    new_img.save(dest_img)
    new_mask.save(dest_mask)
    




In [61]:
def zoom_center(img, zoom_factor=1.5,name_img = None, mask=None ,dest_img=None, dest_mask=None):
    # read original 
    path = os.path.join("..","original-dataset","images",name_img)
    #read image file
    img = cv2.imread(path)

    (h, w) = img.shape[:2]
 
    # calculate the center of the image
    center = (w / 2, h / 2)
    
    
    # perform the rotation and translate the image to its center
    M = cv2.getRotationMatrix2D(center, 360, 1.5) # rotate 360 degrees clockwise around center of image 

    rotated = cv2.warpAffine(img, M, (w, h)) # translate rotated image to its center 
    
    
    #cv2.imshow("Zoom", rotated)
    #cv2.waitKey(0)

    #Save
    dest_img= os.path.join("..","car-segmentation","images","zoom_"+name_img)
    cv2.imwrite(dest_img,rotated)
    dest_mask = os.path.join(dest_mask,"zoom_"+name_img)
    #rotated.save(dest_img)
    mask.save(dest_mask)
    
 


In [62]:
def zoom_left(img, zoom_factor=1.5,name_img = None,mask=None, dest_img=None, dest_mask=None):
    # read original 
    path = os.path.join("..","original-dataset","images",name_img)
    #read image file
    img = cv2.imread(path)
    
    (h, w) = img.shape[:2]
 
    # calculate the left of the image
    left = (w / 4, h / 4)
    
    # perform the rotation and translate the image to its left
    M1 = cv2.getRotationMatrix2D(left, 360, 1.5) # rotate 360 degrees clockwise around left of image 
   # rotated = cv2.warpAffine(img, M, (w, h)) # translate rotated image to its center 
    rotated1 = cv2.warpAffine(img, M1, (w, h)) # translate rotated image to its center 
    #cv2.imshow("Rotated (Correct)", rotated1)
    #cv2.waitKey(0)


    #Save
    dest_img = os.path.join("..","car-segmentation","images","zoom_"+name_img)
    cv2.imwrite(dest_img, rotated1)
    dest_mask = os.path.join(dest_mask,"zoom_"+name_img)
    #rotated1.save(dest_img)
    mask.save(dest_mask)

In [65]:
#Get Directory of car-segmentation images
parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) #parent directory
img_folder = os.path.join(parent, "original-dataset", "images")
masks_folder = os.path.join(parent, "original-dataset", "masks")
destination_img_folder = os.path.join(parent, "car-segmentation", "images") #cartella di destinazione su cui salvare le immagini
destination_mask_folder = os.path.join(parent, "car-segmentation", "masks") #cartella di destinazione su cui salvare le maschere 
all_images = os.listdir(img_folder)
all_masks = os.listdir(masks_folder)

for i in range(5): #len(all_images)
    print("Eseguo sull'immagine: ", i+1)
    #immagine
    file = all_images[i] #Nome dell'immagine (e della maschera)
    image_path = os.path.join(img_folder, file)
    img = Image.open(image_path)
    #maschera
    mask_path = os.path.join(masks_folder,file)
    mask = Image.open(mask_path)
    
    #Operazioni
    #1) Mirror orizzonale di ogni immagine
    mirror_hor(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #2) Rumore gaussiano
    gaussian_noise(image_path,file,mask,destination_img_folder,destination_mask_folder)
    
    #3) Colore
    color(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #4) Blur
    blur(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #5) Rotate 
    rotate(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #6) scale
    zoom_center(img, zoom_factor=1.5,name_img=file, mask=mask, dest_img=destination_img_folder, dest_mask=destination_mask_folder)
    zoom_left(img, zoom_factor=1.5,name_img=file, mask=mask, dest_img=destination_img_folder, dest_mask=destination_mask_folder)
    
    #7) traslazione
    #traslate(img,file,mask,destination_img_folder,destination_mask_folder)
  

Eseguo sull'immagine:  1
Eseguo sull'immagine:  2
Eseguo sull'immagine:  3
Eseguo sull'immagine:  4



(eom:215658): EOM-CRITICAL **: 15:27:20.899: eom_list_store_get_pos_by_image: assertion 'EOM_IS_IMAGE (image)' failed


Eseguo sull'immagine:  5



** (gdk-pixbuf-thumbnailer:216016): WARNING **: 15:27:57.472: Could not thumbnail 'file:///tmp/.mate_desktop_thumbnail.9C00Z1': Apertura del file «/tmp/.mate_desktop_thumbnail.9C00Z1» non riuscita: File o directory non esistente

** (gdk-pixbuf-thumbnailer:216025): WARNING **: 15:27:57.718: Could not thumbnail 'file:///tmp/.mate_desktop_thumbnail.VF7H01': Apertura del file «/tmp/.mate_desktop_thumbnail.VF7H01» non riuscita: File o directory non esistente

(eom:215658): GLib-GObject-CRITICAL **: 15:27:57.720: g_object_unref: assertion 'G_IS_OBJECT (object)' failed

** (gdk-pixbuf-thumbnailer:216082): WARNING **: 15:27:59.295: Could not thumbnail 'file:///tmp/.mate_desktop_thumbnail.OHB7Z1': Apertura del file «/tmp/.mate_desktop_thumbnail.OHB7Z1» non riuscita: File o directory non esistente

(eom:215658): GLib-GObject-CRITICAL **: 15:27:59.301: g_object_unref: assertion 'G_IS_OBJECT (object)' failed

(eom:215658): GLib-GObject-CRITICAL **: 15:27:59.301: g_object_unref: assertion 'G_IS_O